In [ ]:
!git clone https://github.com/tias112/table-transformer.git
%cd table-transformer


Cloning into 'table-transformer'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (532/532), done.
remote: Compressing objects: 100% (386/386), done.
remote: Total 532 (delta 311), reused 309 (delta 136), pack-reused 0
Receiving objects: 100% (532/532), 190.16 KiB | 3.22 MiB/s, done.
Resolving deltas: 100% (311/311), done.
/content/table-transformer


In [ ]:
%cd table-transformer
!pip install -r requirements.txt
import os
#os.kill(os.getpid(), 9)  ## restart runtime

/content/table-transformer


In [ ]:
%cd table-transformer
!pip install --upgrade numpy
#os.kill(os.getpid(), 9)  ## restart runtime

[Errno 2] No such file or directory: 'table-transformer'
/content/table-transformer
     |████████████████████████████████| 15.7 MB 248 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.17.0
    Uninstalling numpy-1.17.0:
      Successfully uninstalled numpy-1.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.21.3 which is incompatible.
xarray 0.18.2 requires pandas>=1.0, but you have pandas 0.25.3 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.0 which is incompatible.
tensorflow 2.8.0 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.0.1 which is incompatible.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.

**Download data set and model weights**

In [ ]:
############   Examples from BDOC  #############
from google.colab import drive
import zipfile
drive.mount('/content/gdrive')
%cd table-transformer
!cp '/content/gdrive/MyDrive/for github/model_11.pth' model.pth
!cp '/content/gdrive/MyDrive/for github/pubtables1m_detection_detr_r18.pth' pubtables1m_detection_detr_r18.pth

%mkdir data
%cd data
!cp '/content/gdrive/MyDrive/for github/nc_ttv.zip' nc_ttv.zip
%mkdir nc_ttv
with zipfile.ZipFile('nc_ttv.zip', 'r') as zip_ref:
    zip_ref.extractall('./nc_ttv/')
%cd ..

Mounted at /content/gdrive
/content/table-transformer
/content/table-transformer/data
/content/table-transformer


**Test data set with 1 image**

In [ ]:

%cd table-transformer
!cp '/content/gdrive/MyDrive/for github/val.zip' val.zip
with zipfile.ZipFile('val.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/nc_ttv/val')
!cp '/content/gdrive/MyDrive/for github/nc_ttv_1.zip' nc_ttv_1.zip
%mkdir ./data/nc_ttv_1
with zipfile.ZipFile('nc_ttv_1.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/nc_ttv_1/')
!cp -r ./data/nc_ttv/val/words ./data/nc_ttv_1/val


mkdir: cannot create directory ‘./data/nc_ttv_1’: File exists


In [ ]:
%cd table-transformer
!git pull

/content/table-transformer
Already up to date.


In [ ]:
import sys
sys.argv=['']
del sys

In [ ]:
%cd table-transformer/
!git pull

/content/table-transformer
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/tias112/table-transformer
   9312271..f2672d3  main       -> origin/main
Updating 9312271..f2672d3
Fast-forward
 core.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


*******************************************************************************
**Structure dataset**

BDTablesDataset provides data set in a format applicable for Table Transformer:
- process bboxes
- image cropping (crop table)
- image padding (add padding 30px with white spaces - required by TT)
- extract texts (OCR)

In [ ]:
from bd_dataset import BDTablesDataset

category_map = {
        "table": 0,
        "Cell": 1,
        "header": 2
    }
ds = BDTablesDataset(root='./data/nc_ttv/val',
                         do_crop=True,
                         do_padding=True, 
                     category_map = category_map)
ds._process_images(227)
print(ds._get_cropped_bbox('novbig_10_00021.png'))

processed: dict_keys(['novheader_sep_bottom_up_seps_2_1_00105.png', 'novbig_3_mpp_00009.png', 'novheader_sep_2_train_header_sep_2_00000.png', 'novbig_6_2_mpp_00006.png', 'novbig_10_00011.png', 'nov3_simple_borderless_00014.png', 'novbig_9_mpp_00079.png', 'novmultiline_cells_3_00002.png', 'novsimple_4_00007.png', 'novmultiline_cells_3_00004.png', 'novbig_7_mpp_00024.png', 'novmultiline_cells_2_00008.png', 'nov1_simple_borderless_00023.png', 'novheader_sep_4_copied_header_sep_4_00005.png', 'novheader_sep_4_copied_header_sep_4_00139.png', 'novsimple_2_00010.png', 'novcomplex_5_00004.png', 'novcomplex_5_00013.png', 'novtext_structure_changes_3_00006.png', 'novbig_7_mpp_00036.png', 'novheader_sep_2_train_header_sep_2_00062.png', 'novbig_6_1_mpp_00030.png', 'novheader_sep_2_val_header_sep_2_00047.png', 'novtext_structure_changes_3_00012.png', 'novbig_7_mpp_00026.png', 'novheader_sep_4_renamed_to_5_00099.png', 'novbig_9_mpp_00048.png', 'nov3_00023.png', 'novbig_13_00007.png', 'novbig_14_00025

**Run structure recognition based on BD dataset**

In [ ]:
from core import TableRecognizer
import json 
m = TableRecognizer(checkpoint_path="model.pth",
                    make_coco=True,
                    export_objects=False,
                    for_analysis=True,
                    save_debug_images=True,
                    root = "./data/nc_ttv/val",
                    config_file="./src/structure_config.json",
                    images_dir="processed",
                    data_type="structure",
                    ds = ds                  
                    )

m.process(195)
#m.process(3)
#for filename in lines:
    #output = m.predict(os.path.join(image_directory, filename))


Streaming output truncated to the last 5000 lines.
        [ 38.7092, 624.1080, 691.6666, 645.4470],
        [633.3505,  90.4879, 691.4063, 168.1894],
        [424.9787,  39.4722, 692.0217,  64.4216],
        [ 38.5578, 702.8405, 691.8579, 722.8389],
        [ 38.7881, 640.4034, 691.6337, 660.9911],
        [380.5709, 371.4674, 472.2200, 411.3426],
        [446.9962,  65.7550, 501.8653,  97.7530],
        [ 37.9613, 689.9034, 692.2283, 709.4334],
        [269.1749,  39.0586, 299.9225, 924.6136],
        [ 38.4227, 546.8237, 692.0391, 567.5170],
        [ 38.9385, 495.8439, 691.5756, 516.3749],
        [352.1328,  40.2541, 404.9240,  94.8301],
        [653.2496,  38.9656, 692.5347, 925.0715],
        [420.0395,  42.8420, 455.0028, 921.7827],
        [ 39.1924, 781.2162, 691.2843, 801.1512],
        [ 39.0268, 208.2537, 691.3088, 228.8092],
        [ 38.0927, 188.5875, 270.5332, 234.6563],
        [ 38.9464, 364.9843, 691.5687, 385.4078],
        [ 36.5522, 876.4960, 692.1470, 923.7112],

*************
**Detection dataset**

In [ ]:
from bd_dataset import BDTablesDataset

category_map = {
        "table": 0,
        "Cell": 1,
        "header": 2
    }
ds = BDTablesDataset(root='./data/nc_ttv/val',
                         do_crop=False,
                         do_padding=False, 
                     category_map = category_map)

ds._process_images(227)


processed: dict_keys(['novheader_sep_bottom_up_seps_2_1_00105.png', 'novbig_3_mpp_00009.png', 'novheader_sep_2_train_header_sep_2_00000.png', 'novbig_6_2_mpp_00006.png', 'novbig_10_00011.png', 'nov3_simple_borderless_00014.png', 'novbig_9_mpp_00079.png', 'novmultiline_cells_3_00002.png', 'novsimple_4_00007.png', 'novmultiline_cells_3_00004.png', 'novbig_7_mpp_00024.png', 'novmultiline_cells_2_00008.png', 'nov1_simple_borderless_00023.png', 'novheader_sep_4_copied_header_sep_4_00005.png', 'novheader_sep_4_copied_header_sep_4_00139.png', 'novsimple_2_00010.png', 'novcomplex_5_00004.png', 'novcomplex_5_00013.png', 'novtext_structure_changes_3_00006.png', 'novbig_7_mpp_00036.png', 'novheader_sep_2_train_header_sep_2_00062.png', 'novbig_6_1_mpp_00030.png', 'novheader_sep_2_val_header_sep_2_00047.png', 'novtext_structure_changes_3_00012.png', 'novbig_7_mpp_00026.png', 'novheader_sep_4_renamed_to_5_00099.png', 'novbig_9_mpp_00048.png', 'nov3_00023.png', 'novbig_13_00007.png', 'novbig_14_00025

In [ ]:

from core import TableRecognizer
import json 
m = TableRecognizer(checkpoint_path="pubtables1m_detection_detr_r18.pth",
                    make_coco=True,
                    export_objects=False,
                    for_analysis=True,
                    save_debug_images=True,
                    root = "./data/nc_ttv/val",
                    config_file="./src/detection_config.json",
                    images_dir="processed",
                    data_type="detection",
                    ds = ds                  
                    )

m.process(195)

{'lr': 5e-05, 'lr_backbone': 1e-05, 'batch_size': 2, 'weight_decay': 0.0001, 'epochs': 20, 'lr_drop': 1, 'lr_gamma': 0.9, 'clip_max_norm': 0.1, 'backbone': 'resnet18', 'num_classes': 2, 'dilation': False, 'position_embedding': 'sine', 'emphasized_weights': {}, 'enc_layers': 6, 'dec_layers': 6, 'dim_feedforward': 2048, 'hidden_dim': 256, 'dropout': 0.1, 'nheads': 8, 'num_queries': 15, 'pre_norm': True, 'masks': False, 'aux_loss': False, 'mask_loss_coef': 1, 'dice_loss_coef': 1, 'ce_loss_coef': 1, 'bbox_loss_coef': 5, 'giou_loss_coef': 2, 'eos_coef': 0.4, 'set_cost_class': 1, 'set_cost_bbox': 5, 'set_cost_giou': 2, 'device': 'cuda', 'seed': 42, 'start_epoch': 0, 'num_workers': 1, '__module__': 'core', '__dict__': <attribute '__dict__' of 'Args' objects>, '__weakref__': <attribute '__weakref__' of 'Args' objects>, '__doc__': None, 'data_type': 'detection'}
----------------------------------------------------------------------------------------------------
loading model
loading model from 

[{'label': 3, 'bbox': [181, 286, 2446, 608]}]


In [ ]:
import shutil
#shutil.make_archive("words", 'zip', './data/nc_ttv/val/words/lines')
shutil.make_archive("processed", 'zip', './data/nc_ttv/val/processed')
shutil.make_archive("debug-images", 'zip', './data/nc_ttv/val/debug')

'/content/table-transformer/debug-images.zip'